<a href="https://colab.research.google.com/github/elizabethyoo/team_9/blob/data%2FBFS_mf/Complie_sbp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [25]:
import pandas as pd
import glob
import re
import numpy as no
import seaborn as sns
import matplotlib.pyplot as plt

In [52]:
files = glob.glob(f"/content/drive/MyDrive/Group 9 Project Documents/Datasets/business_pulse/*.xlsx") 

In [50]:
emp_files = [x for x in files if 'empclass' in x]
st_sector_files = [x for x in files if '/national_state_sector' in x]
st_files = [x for x in files if '/national_state_' in x and not '/national_state_sector' in x]
naics3_files = [x for x in files if 'naics3' in x]
indexes_files = [x for x in files if 'indexes' in x]
msa_files = [x for x in files if 'top_50_msa' in x]
response_urr_files = [x for x in files if 'response_urr' in x]

In [75]:
st_sector_files[0]

'/content/drive/MyDrive/Group 9 Project Documents/Datasets/business_pulse/national_state_sector_26Apr20_02May20.xlsx'

In [76]:
regex = r'\d+[A-Z][a-z]+\d+_\d+[A-Z][a-z]+\d+'  
match = re.search(regex, st_sector_files[0].split('/')[-1])[0]
print(match)

26Apr20_02May20


In [77]:
st_df = pd.read_excel(st_sector_files[0]).assign(DATE=match)

In [78]:
st_df.head()

,ST,NAICS_SECTOR,INSTRUMENT_ID,QUESTION,ANSWER_ID,ANSWER_TEXT,ESTIMATE_PERCENTAGE,SE,DATE
0,-,-,1,"Overall, how has this business been affected b...",1,Large negative effect,51.4%,0.21%,26Apr20_02May20
1,-,-,1,"Overall, how has this business been affected b...",2,Moderate negative effect,38.5%,0.27%,26Apr20_02May20
2,-,-,1,"Overall, how has this business been affected b...",3,Little or no effect,7.6%,0.29%,26Apr20_02May20
3,-,-,1,"Overall, how has this business been affected b...",4,Moderate positive effect,1.7%,0.08%,26Apr20_02May20
4,-,-,1,"Overall, how has this business been affected b...",5,Large positive effect,0.8%,0.06%,26Apr20_02May20
